## MNIST 손글씨 숫자 분류: 딥러닝 첫걸음

이 노트북은 가장 대표적인 딥러닝 예제인 MNIST 손글씨 숫자 데이터셋을 사용하여, 기본적인 완전 연결 신경망(Dense Neural Network)을 구축하고 훈련시키는 전체 과정을 다룹니다.

### 1. 라이브러리 임포트 및 환경 설정
필요한 라이브러리를 불러오고, 재현성을 위해 텐서플로우의 랜덤 시드를 고정합니다.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras import models, layers

# 랜덤 시드 고정
tf.random.set_seed(1234)

### 2. 데이터 로드 및 확인
Keras에 내장된 MNIST 데이터셋을 로드하고, 훈련 데이터와 테스트 데이터의 형태(shape)를 출력하여 구조를 확인합니다.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print("--- 훈련 데이터 ---")
print("이미지 형태:", train_images.shape)
print("레이블 형태:", train_labels.shape)

print("--- 테스트 데이터 ---")
print("이미지 형태:", test_images.shape)
print("레이블 형태:", test_labels.shape)

### 3. 데이터 전처리
신경망에 데이터를 입력하기 전에 두 가지 주요 전처리 작업을 수행합니다.

1.  **차원 변환 (Reshaping)**: 2차원 이미지 데이터(28x28 픽셀)를 1차원 벡터(784)로 펼칩니다. 이는 `Dense` 레이어에 입력하기 위함입니다.
2.  **정규화 (Normalization/Scaling)**: 픽셀 값의 범위를 0-255에서 0-1 사이로 조정합니다. 이는 모델의 학습을 더 안정적이고 빠르게 만듭니다.

In [ ]:
# 훈련 이미지 전처리
train_images_flat = train_images.reshape(train_images.shape[0], 28 * 28)
train_images_scaled = train_images_flat.astype('float32') / 255

# 테스트 이미지 전처리
test_images_flat = test_images.reshape(test_images.shape[0], 28 * 28)
test_images_scaled = test_images_flat.astype('float32') / 255

print("전처리 후 훈련 이미지 형태:", train_images_scaled.shape)

### 4. 신경망 모델 구축
`keras.Sequential`을 사용하여 모델을 순차적으로 구성합니다.

- **은닉층 (Hidden Layers)**: `relu` 활성화 함수를 사용하는 여러 개의 `Dense` 층을 추가합니다.
- **출력층 (Output Layer)**: 0부터 9까지 10개의 숫자를 분류해야 하므로, 10개의 뉴런을 가진 `Dense` 층을 사용합니다. 각 뉴런이 해당 숫자의 확률을 나타내도록 `softmax` 활성화 함수를 사용합니다.

In [ ]:
model = keras.Sequential([
    # 입력 데이터의 형태는 첫 번째 레이어에서 자동으로 추론됩니다.
    layers.Dense(256, activation='relu'),
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    # 출력층: 10개의 클래스에 대한 확률을 출력
    layers.Dense(10, activation='softmax')
])

### 5. 모델 컴파일
모델을 훈련하기 전에 학습 프로세스를 설정합니다.

- **Optimizer**: `rmsprop` 최적화 알고리즘을 사용합니다.
- **Loss Function**: `sparse_categorical_crossentropy`를 손실 함수로 사용합니다. 이 함수는 레이블이 정수 형태일 때 별도의 원-핫 인코딩 없이 사용할 수 있어 편리합니다.
- **Metrics**: 훈련 및 평가 과정에서 `accuracy`(정확도)를 모니터링합니다.

In [ ]:
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


### 6. 모델 훈련
`fit()` 메서드를 사용하여 모델을 훈련시킵니다.

- `epochs`: 전체 훈련 데이터셋을 몇 번 반복하여 학습할지 결정합니다.
- `batch_size`: 한 번에 메모리에 올릴 데이터의 양을 지정합니다. 이 배치 단위로 가중치 업데이트가 일어납니다.

In [ ]:
history = model.fit(
    train_images_scaled,  # 전처리된 훈련 이미지
    train_labels,         # 훈련 레이블
    epochs=100,
    batch_size=128,
    validation_split=0.2  # 훈련 데이터의 20%를 검증용으로 사용
)

### 7. 모델 평가
`evaluate()` 메서드를 사용하여 훈련된 모델의 성능을 훈련 데이터셋과 테스트 데이터셋에 대해 각각 평가하고, 최종 손실(loss)과 정확도(accuracy)를 확인합니다.

In [ ]:
print("\n--- 모델 평가 ---")
train_loss, train_acc = model.evaluate(train_images_scaled, train_labels)
print(f"훈련셋 손실: {train_loss:.4f}, 정확도: {train_acc:.4f}")

test_loss, test_acc = model.evaluate(test_images_scaled, test_labels)
print(f"테스트셋 손실: {test_loss:.4f}, 정확도: {test_acc:.4f}")

### 8. 모델 구조 확인
`summary()` 메서드를 통해 모델의 전체 구조와 각 층의 파라미터 수를 확인할 수 있습니다.

In [ ]:
model.summary()